# Conformación dataset

In [5]:
import glob
import xarray as xr
import geopandas as gpd
from geopandas.tools import sjoin

In [6]:
# Ruta al archivo shape
shapefile_path = "../shapes/shape_boyaca_municipios.shp"

# Carpeta que contiene los archivos .nc (Descargando usando el notebook 1.get_data.ipynb)
nc_folder_path = "..data/archivos_climate_serv/"

# Cargar el archivo shape
gdf = gpd.read_file(shapefile_path)

# Obtener las coordenadas extremas iniciales
min_latitude = float('inf')
max_latitude = float('-inf')
min_longitude = float('inf')
max_longitude = float('-inf')

# Iterar sobre los polígonos del archivo shape
for i, geometry in enumerate(gdf.geometry):
    # Obtener los límites de latitud y longitud del polígono actual
    polygon_bounds = geometry.bounds
    
    # Actualizar los valores máximos y mínimos de latitud y longitud
    min_latitude = min(min_latitude, polygon_bounds[1])
    max_latitude = max(max_latitude, polygon_bounds[3])
    min_longitude = min(min_longitude, polygon_bounds[0])
    max_longitude = max(max_longitude, polygon_bounds[2])

# min_longitude -= 0.001
# max_longitude += 0.001

# Crear una lista para almacenar los datos filtrados
filtered_datasets = []

# Obtener la lista de archivos .nc en la carpeta
nc_file_paths = glob.glob(nc_folder_path + "*.nc")

# Iterar sobre los archivos .nc
for nc_file_path in nc_file_paths:
    # Leer el archivo .nc utilizando xarray
    ds = xr.open_dataset(nc_file_path)
    
    # Filtrar por latitud y longitud máximas y mínimas
    ds_filtered = ds.sel(
        longitude=slice(min_longitude, max_longitude),
        latitude=slice(min_latitude, max_latitude)
    )
    
    # Agregar el dataset filtrado a la lista
    filtered_datasets.append(ds_filtered)

# Combinar los datasets filtrados en uno solo
combined_dataset = xr.concat(filtered_datasets, dim="time")

# Ruta para guardar el archivo .nc combinado
output_nc_path = "../data/data_Boyaca.nc"

# Guardar el dataset combinado en un nuevo archivo .nc
combined_dataset.to_netcdf(output_nc_path)

In [21]:
# Cargar el archivo .nc utilizando xarray
data_boyaca = xr.open_dataset('../data/data_Boyaca.nc')
df_boyaca = data_boyaca.to_dataframe().reset_index()

In [22]:
# Ruta al archivo shape
shapefile_path = "../shapes/shape_boyaca_municipios.shp"

# Cargar el archivo shape
shape_boyaca_municipios = gpd.read_file(shapefile_path)

In [23]:
gdf_boyaca = gpd.GeoDataFrame(df_boyaca, geometry=gpd.points_from_xy(df_boyaca.longitude, df_boyaca.latitude))

In [24]:
# gdf_boyaca = gdf_boyaca.to_crs(epsg=4686)
joined_data = sjoin(gdf_boyaca, shape_boyaca_municipios, how='left', predicate='within')

In [25]:
joined_data.head()

,latitude,longitude,time,precip,geometry,index_right,municipio
0,4.674999,-74.625008,1981-01-01,4.838065,POINT (-74.62501 4.67500),NaN,NaN
1,4.674999,-74.625008,1981-01-02,1.995213,POINT (-74.62501 4.67500),NaN,NaN
2,4.674999,-74.625008,1981-01-03,0.000000,POINT (-74.62501 4.67500),NaN,NaN
3,4.674999,-74.625008,1981-01-04,0.000000,POINT (-74.62501 4.67500),NaN,NaN
4,4.674999,-74.625008,1981-01-05,0.000000,POINT (-74.62501 4.67500),NaN,NaN


In [26]:
# Valores nulo
joined_data_null = joined_data[joined_data['municipio'].isnull()]

# valores no nulos
joined_data_not_null = joined_data[joined_data['municipio'].notnull()]

In [27]:
joined_data_null.shape, joined_data_not_null.shape, joined_data.shape

((27635944, 7), (11773160, 7), (39409104, 7))

In [28]:
joined_data_not_null.sample()

,latitude,longitude,time,precip,geometry,index_right,municipio
4499340,4.924999,-73.375008,2000-01-12,0.0,POINT (-73.37501 4.92500),29.0,Macanal


In [29]:
# Eliminar la columna index_right y geometry
joined_data_not_null.drop(columns=['index_right', 'geometry'], inplace=True)

C:\Users\BPS-EYDUARTE\AppData\Local\Temp\ipykernel_18880\877626945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_data_not_null.drop(columns=['index_right', 'geometry'], inplace=True)


In [16]:
# Exportar a csv los datos no nulos
joined_data_not_null.to_csv('../data/data_boyaca.csv', index=False)

In [30]:
# Exportar a .nc el dataset
ds = xr.Dataset.from_dataframe(joined_data_not_null)

file_nc = "../data/data_boyaca_municipios.nc"

ds.to_netcdf(file_nc)